In [ ]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import json

In [ ]:
__all__ = ['ResNet_StoDepth_lineardecay', 'resnet18_StoDepth_lineardecay', 'resnet34_StoDepth_lineardecay', 'resnet50_StoDepth_lineardecay', 'resnet101_StoDepth_lineardecay',
           'resnet152_StoDepth_lineardecay']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    # """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    # """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class StoDepth_BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, prob, multFlag, inplanes, planes, stride=1, downsample=None):
        super(StoDepth_BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        self.prob = prob
        self.m = torch.distributions.bernoulli.Bernoulli(torch.Tensor([self.prob]))
        self.multFlag = multFlag

    def forward(self, x):
        
        identity = x.clone()

        if self.training:
            if torch.equal(self.m.sample(),torch.ones(1)):

                self.conv1.weight.requires_grad = True
                self.conv2.weight.requires_grad = True

                out = self.conv1(x)
                out = self.bn1(out)
                out = self.relu(out)
                out = self.conv2(out)
                out = self.bn2(out)

                if self.downsample is not None:
                    identity = self.downsample(x)

                out += identity
            else:
                # Resnet does not use bias terms
                self.conv1.weight.requires_grad = False
                self.conv2.weight.requires_grad = False
                
                if self.downsample is not None:
                    identity = self.downsample(x)

                out = identity
        else:
            

            out = self.conv1(x)
            out = self.bn1(out)
            out = self.relu(out)
            out = self.conv2(out)
            out = self.bn2(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            if self.multFlag:
                out = self.prob*out + identity
            else:
                out = out + identity

        out = self.relu(out)

        return out


class StoDepth_Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, prob, multFlag, inplanes, planes, stride=1, downsample=None):
        super(StoDepth_Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.prob = prob
        self.m = torch.distributions.bernoulli.Bernoulli(torch.Tensor([self.prob]))
        self.multFlag = multFlag

    def forward(self, x):

        identity = x.clone()

        if self.training:
            if torch.equal(self.m.sample(),torch.ones(1)):
                self.conv1.weight.requires_grad = True
                self.conv2.weight.requires_grad = True
                self.conv3.weight.requires_grad = True

                out = self.conv1(x)
                out = self.bn1(out)
                out = self.relu(out)

                out = self.conv2(out)
                out = self.bn2(out)
                out = self.relu(out)

                out = self.conv3(out)
                out = self.bn3(out)

                if self.downsample is not None:
                    identity = self.downsample(x)

                out += identity
            else:
                # Resnet does not use bias terms
                self.conv1.weight.requires_grad = False
                self.conv2.weight.requires_grad = False
                self.conv3.weight.requires_grad = False

                if self.downsample is not None:
                    identity = self.downsample(x)

                out = identity
        else:
            out = self.conv1(x)
            out = self.bn1(out)
            out = self.relu(out)

            out = self.conv2(out)
            out = self.bn2(out)
            out = self.relu(out)

            out = self.conv3(out)
            out = self.bn3(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            if self.multFlag:
                out = self.prob*out + identity
            else:
                out = out + identity

        out = self.relu(out)

        return out


class ResNet_StoDepth_lineardecay(nn.Module):

    def __init__(self, block, prob_0_L, multFlag, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet_StoDepth_lineardecay, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.multFlag = multFlag
        self.prob_now = prob_0_L[0]
        self.prob_delta = prob_0_L[0]-prob_0_L[1]
        self.prob_step = self.prob_delta/(sum(layers)-1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, StoDepth_lineardecayBottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, StoDepth_lineardecayBasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.prob_now, self.multFlag, self.inplanes, planes, stride, downsample))
        self.prob_now = self.prob_now - self.prob_step
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.prob_now, self.multFlag, self.inplanes, planes))
            self.prob_now = self.prob_now - self.prob_step

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18_StoDepth_lineardecay(pretrained=False, prob_0_L=[1,0.5], multFlag=True, **kwargs):
    # """Constructs a ResNet_StoDepth_lineardecay-18 model.
    # Args:
    #     pretrained (bool): If True, returns a model pre-trained on ImageNet
    # """
    model = ResNet_StoDepth_lineardecay(StoDepth_BasicBlock, prob_0_L, multFlag, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34_StoDepth_lineardecay(pretrained=False, prob_0_L=[1,0.5], multFlag=True, **kwargs):
    # """Constructs a ResNet_StoDepth_lineardecay-34 model.
    # Args:
    #     pretrained (bool): If True, returns a model pre-trained on ImageNet
    # """
    model = ResNet_StoDepth_lineardecay(StoDepth_BasicBlock, prob_0_L, multFlag, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


# UBAH JADI multFlag=False 
def resnet50_StoDepth_lineardecay(pretrained=False, prob_0_L=[1,0.5], multFlag=True, **kwargs):
    # """Constructs a ResNet_StoDepth_lineardecay-50 model.
    # Args:
    #     pretrained (bool): If True, returns a model pre-trained on ImageNet
    # """
    model = ResNet_StoDepth_lineardecay(StoDepth_Bottleneck, prob_0_L, multFlag, [3, 4, 6, 3], **kwargs)
    if pretrained:
        print("Program Pretrained",pretrained)
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101_StoDepth_lineardecay(pretrained=False, prob_0_L=[1,0.5], multFlag=True, **kwargs):
    # """Constructs a ResNet_StoDepth_lineardecay-101 model.
    # Args:
    #     pretrained (bool): If True, returns a model pre-trained on ImageNet
    # """
    model = ResNet_StoDepth_lineardecay(StoDepth_Bottleneck, prob_0_L, multFlag, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152_StoDepth_lineardecay(pretrained=False, prob_0_L=[1,0.5], multFlag=True, **kwargs):
    # """Constructs a ResNet_StoDepth_lineardecay-152 model.
    # Args:
    #     pretrained (bool): If True, returns a model pre-trained on ImageNet
    # """
    model = ResNet_StoDepth_lineardecay(StoDepth_Bottleneck, prob_0_L, multFlag, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model


def test():
    net = resnet34_StoDepth_lineardecay()
    x = torch.randn(2,3,224,224)
    y = net(x)



In [ ]:
# %ls drive/MyDrive/

model = torch.load('drive/MyDrive/ResNetModel/myResNetStokastik404-0.9418.model')

In [ ]:
model.eval()

In [ ]:
print('==> Preparing dataset..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='drive/MyDrive/MyThesis/cifar-10-batches-py', train=True, download=True, transform=transform_train)


testset = torchvision.datasets.CIFAR10(
    root='drive/MyDrive/MyThesis/cifar-10-batches-py', train=False, download=True, transform=transform_test)


trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=4, shuffle=True, num_workers=2)    

testloader = torch.utils.data.DataLoader(
    testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing dataset..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# print(model) 
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9,weight_decay=1e-4,nesterov=True,dampening=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
for i,child in enumerate(model.children()):
    print(i, "======" ,child)

In [ ]:
# newModel = []
# child_of_7 = []
# for i,child in enumerate(model.children()):
#     if i <= 6:
#       newModel.append(child)
#     elif i == 7:
#       for j,data2 in enumerate(child.children()):
#           child_of_7.append(data2)
#     elif i > 7:
#       newModel.append(child)
  

# # print("==========================")
# child_of_7_m = torch.nn.Sequential(child_of_7[0],child_of_7[1],child_of_7[2])
# # print("==========================")

# insert_at = 7  # Index at which you want to insert item
# newModel_b = newModel[:]   # Created copy of list "a" as "b".
# newModel_b[insert_at:insert_at] =  torch.nn.Sequential(child_of_7_m)

In [ ]:
# class resnet_remove_layer(nn.Module):
#     def __init__(self):
#         super().__init__()
#         r_mode = model
#         self.Conv1 = torch.nn.Sequential(*(list(r_mode.children())[0:7]))
#         self.Conv2 = model.layer4[0] , model.layer4[1] , model.layer4[2] ,  torch.nn.Sequential(*(list(r_mode.children())[8:10]))
#         # self.Conv3 = model.layer4[1]
#         # self.Conv4 = model.layer4[2]
#         # self.Conv5 = torch.nn.Sequential(*(list(r_mode.children())[8:10]))
   
#     def forward(self,x):
#         out1 = self.Conv1(x)
#         out2 = self.Conv2(out1)
#         # out3 = self.Conv3(out2)
#         # out4 = self.Conv4(out3)
#         # out5 = self.Conv5(out2)
        
#         return x

# net2 = resnet_remove_layer()
# print(net2)


# **TRAINING FUNCTION**

In [ ]:
info_train = []
info_test = []

def train(epoch,end_epoch):
    best_acc = 0.9
    correct = 0
    total = 0
    running_loss = 0.0
    running_corrects = 0

    model.train()
    print("Epoch: ",epoch+1,"||",end_epoch)
    with torch.set_grad_enabled(True):
     for inputs, labels in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # correct += (predicted == labels).sum().item()
        running_corrects += (predicted == labels.data).sum().item()
        running_loss += loss.item()

        epoch_loss = running_loss / total
        epoch_acc = running_corrects / total

    info_train.append({'epoch':epoch+1,'loss':epoch_loss,'acc':epoch_acc})    
    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        'Training = ', epoch_loss, epoch_acc))
    
    test_epoch()

    if epoch_acc > best_acc:
      acc_format_name = format(epoch_acc, ".4f")
      print("saved : ",acc_format_name)
      torch.save(model,f'drive/MyDrive/ResNetModel/myResNetStokastik{epoch+1}-{acc_format_name}.model')

# TEST **FUNCTION**

In [ ]:
def test_epoch(model):
    test_running_corrects = 0
    test_running_loss = 0.0
    total_test = 0
    # model.eval()

    with torch.no_grad():
      for inputs_test,labels_test in testloader:
          inputs_test = inputs_test.to(device)
          labels_test = labels_test.to(device)
          # print("check ==",inputs_test)
          outputs_test = model(inputs_test)
          loss_test = criterion(outputs_test, labels_test)
          _, pred_test = torch.max(outputs_test.data, 1)
          total_test += labels_test.size(0)
          
          test_running_corrects += (pred_test == labels_test).sum().item()
          test_running_loss += loss_test.item()

          # print(test_running_corrects)
          # print(total_test)

          test_epoch_loss = test_running_loss / total_test
          test_epoch_acc = test_running_corrects / total_test

      # info_test.append({'epoch':epoch+1,'loss':test_epoch_loss,'acc':test_epoch_acc})

      params = count_parameters(model)
      # print(test_epoch_loss," ",test_epoch_acc)
      print('{} Loss: {:.4f} Acc: {:.4f} Params: {}'.format(
        'Test =', test_epoch_loss, test_epoch_acc,params))
          

# **START TRAINING AND TEST**

In [ ]:
# start_epoch = 401
# end_epoch = 500

# import time
# since = time.time()

# for epoch in range(start_epoch,end_epoch):
#     train(epoch,end_epoch)

# print('Finished Training and Testing Process')
# time_elapsed = time.time() - since
# print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))

In [ ]:
# import json
# import torch
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()           
# drive = GoogleDrive(gauth)


# with open('info_train.txt', 'w') as json_file:
#     json.dump(info_train, json_file)

# with open('info_test.txt', 'w') as json_file:
#     json.dump(info_test, json_file)

In [ ]:
from torchsummary import summary
# summary(resnet, (3, 224, 224),print_summary=True)
# print(count(resnet))
# resnet.count_params()

from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    # print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
params = count_parameters(model)
print(params)

Total Trainable Params: 12911690
12911690


In [ ]:
model

In [ ]:
import copy
model_new = copy.deepcopy(model)
def test_remove_layer():
    model_new = copy.deepcopy(model)
    
    print("Without Removed Layer")
    result_5 = test_epoch(model)
    

    print("CONV_4_2 Removed")
    del model_new.layer4[2]
    result_1 = test_epoch(model_new)


    print("CONV_4_1 Removed")
    del model_new.layer4[1]
    result_2 = test_epoch(model_new)

    
    print("CONV_3_5 Removed")
    del model_new.layer3[5]
    result_3 = test_epoch(model_new)

    
    print("CONV_3_4 Removed")
    del model_new.layer3[4]
    result_4 = test_epoch(model_new)


    print("CONV_3_3 Removed")
    del model_new.layer3[3]
    result_5 = test_epoch(model_new)

    print("CONV_3_2 Removed")
    del model_new.layer3[2]
    result_6 = test_epoch(model_new)


    print("CONV_3_1 Removed")
    del model_new.layer3[1]
    result_6 = test_epoch(model_new)
   
    # return result_1,result_2,result_3,result_4,result_5

In [ ]:
test_remove_layer()

Without Removed Layer
Total Trainable Params: 12911690
Test = Loss: 0.0879 Acc: 0.8841 Params: 12911690
CONV_4_2 Removed
Total Trainable Params: 12905546
Test = Loss: 0.0880 Acc: 0.8844 Params: 12905546
CONV_4_1 Removed
Total Trainable Params: 8442954
Test = Loss: 0.0883 Acc: 0.8826 Params: 8442954
CONV_3_5 Removed
Total Trainable Params: 7325770
Test = Loss: 0.0896 Acc: 0.8798 Params: 7325770
CONV_3_4 Removed
Total Trainable Params: 7322698
Test = Loss: 0.0971 Acc: 0.8819 Params: 7322698
CONV_3_3 Removed
Total Trainable Params: 7319626
Test = Loss: 0.1328 Acc: 0.8766 Params: 7319626
CONV_3_2 Removed
Total Trainable Params: 6202442
Test = Loss: 0.2294 Acc: 0.8389 Params: 6202442
CONV_3_1 Removed
Total Trainable Params: 5085258
Test = Loss: 0.4714 Acc: 0.2403 Params: 5085258
